In [1]:
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
% matplotlib inline
from six.moves import cPickle
from sklearn.preprocessing import LabelEncoder
from six.moves import cPickle

In [2]:
fpinfo = pd.read_csv('./Data/challenge_Data/BGCON_FPINFO_DATA.csv',
                          encoding = 'utf-16')

In [3]:
fpinfo.head(3)

,CLLT_FP_PRNO,INCB_DVSN,ETRS_YM,FIRE_YM,BEFO_JOB,EDGB,BRCH_CODE
0,6200,R,200306,200407,기타,고졸,25.0
1,2316,R,200104,200904,보험관계인,대학원졸,11.0
2,6207,R,200306,200311,무직,고졸,32.0


In [5]:
fpinfo.describe()

C:\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


,CLLT_FP_PRNO,BRCH_CODE
count,3.152200e+04,3.152000e+04
mean,1.893389e+04,1.621301e+06
std,5.632242e+05,1.262916e+07
min,2.000000e+00,3.000000e+00
25%,7.882250e+03,NaN
50%,1.576250e+04,NaN
75%,2.364275e+04,NaN
max,1.000000e+08,1.000000e+08


In [4]:
fpinfo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31522 entries, 0 to 31521
Data columns (total 7 columns):
CLLT_FP_PRNO    31522 non-null int64
INCB_DVSN       31520 non-null object
ETRS_YM         31522 non-null object
FIRE_YM         31522 non-null object
BEFO_JOB        31266 non-null object
EDGB            31266 non-null object
BRCH_CODE       31520 non-null float64
dtypes: float64(1), int64(1), object(5)
memory usage: 1.7+ MB


In [5]:
string_type = ['INCB_DVSN', 'BEFO_JOB', 'EDGB']
for st in string_type:
    X = fpinfo.loc[~fpinfo[st].isnull(), st].values
    fpinfo.loc[fpinfo[st].isnull(), st] = '-999'
    lbl_enc = LabelEncoder()
    lbl_enc.fit(X)
    for element in lbl_enc.classes_:
        fpinfo.loc[fpinfo[st] == element, st] = lbl_enc.transform(element)
    fpinfo.loc[fpinfo[st] == '-999', st] = -999
    fpinfo[st] = fpinfo[st].astype(np.int64)

In [6]:
conversion = ['ETRS_YM', 'FIRE_YM']
for c in conversion:
    fpinfo.loc[fpinfo[c] == '      ', c] = -999
    fpinfo.loc[fpinfo[c] == '#', c] = -999
    fpinfo[c] = fpinfo[c].astype(np.int64)

In [7]:
def year_convert(input):
    if input < 0:
        return -9999
    else:
        string_type = str(input)

        string_year = string_type[0:4]

        return np.float64(string_year)

def month_convert(input):
    if input < 0 :
        return -9999
    else:
        string_type = str(input)
        string_month = string_type[5:]
        return np.float64(string_month)

def convert(df, colname):
    df[colname+'_Y'] = df[colname].apply(year_convert)
    df[colname+'_M'] = df[colname].apply(month_convert)

    df[colname+'_Y'] =df[colname+'_Y'].astype(np.int64)
    df[colname+'_M'] = df[colname+'_M'].astype(np.int64)
    return df

In [8]:
for colname in ['ETRS_YM', 'FIRE_YM']:
    fpinfo = convert(fpinfo, colname)
    fpinfo.drop(colname, axis = 1, inplace = True)

In [9]:
fpinfo.loc[fpinfo['BRCH_CODE'].isnull(), 'BRCH_CODE'] = -999

In [10]:
cPickle.dump(fpinfo, open('./Dump/fpinfo.pkl', 'wb'), -1)